# DHIS2 Data Download Web API - Central Server V2
## Downloading Health Indicators

In [17]:
import os
import time
import requests
import pandas as pd

In [18]:
WD = '/Users/edinhamzic/Symphony/wb_bangladesh/'
os.chdir(WD)
os.getcwd()

'/Users/edinhamzic/Symphony/wb_bangladesh'

## Organisation Units
### Organisation Unit Levels

In [19]:
organisationUnitsLevels = 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/26/organisationUnitLevels.json?paging=false'
with requests.Session() as s:
    organisationUnitsLevels_json = s.get(organisationUnitsLevels,auth=('wbbigdata_view', 'DGHS@1234#')).json()
pd.DataFrame.from_dict(organisationUnitsLevels_json['organisationUnitLevels'])

,displayName,id
0,Community Clinic and Union Level HF,P3xaO9CNrVY
1,Country,kv8G6cp7G8n
2,District and National/Divisional level HF,TY67fOo5ywC
3,Division,hrLzNYoLILu
4,Level 7,sVti5YayDAf
5,Union and Upazila level HF,IWh6WrGXOyN
6,Upazila and District level HF,WDBz0AfvG8C


### Organisation Unit Groups

In [20]:
organisationUnitGroups = 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/26/organisationUnitGroups.json?paging=false'
with requests.Session() as s:
    organisationUnitGroups_json = s.get(organisationUnitGroups,auth=('wbbigdata_view', 'DGHS@1234#')).json()
organisationUnitGroups_json = pd.DataFrame.from_dict(organisationUnitGroups_json['organisationUnitGroups'])

```
geos = {'District and National_Divisional level HF':'3',
        'Union and Upazila level HF': '5',
        'Upazila and District level HF': '4',
        'Upazila':'XpE4AlQZRkV',
        'Upazila Health Complex':'TwFxBSq8iAg',
        'District': 'PyK69cVqr5l',
        'District Hospital_General Hospital':'yiUFzYnD1f9',
        'District NGO & Private Total': 'Uey6NprfqrE'}
geo_types = {'District and National_Divisional level HF':'LEVEL',
        'Union and Upazila level HF': 'LEVEL',
        'Upazila and District level HF': 'LEVEL',
        'Upazila':'GROUP',
        'Upazila Health Complex':'GROUP',
        'District': 'GROUP',
        'District Hospital_General Hospital':'GROUP',
        'District NGO & Private Total': 'GROUP'
            }
print(geos)
print(geo_types)
```

In [39]:
geos = {'Upazila':'XpE4AlQZRkV',
        'Upazila Health Complex':'TwFxBSq8iAg',
        'District': 'PyK69cVqr5l'}
geo_types = {'Upazila':'GROUP',
             'Upazila Health Complex':'GROUP',
             'District': 'GROUP'}
print(geos)
print(geo_types)

{'Upazila': 'XpE4AlQZRkV', 'Upazila Health Complex': 'TwFxBSq8iAg', 'District': 'PyK69cVqr5l'}
{'Upazila': 'GROUP', 'Upazila Health Complex': 'GROUP', 'District': 'GROUP'}


## Predefined Health Indicators

In [40]:
health_indicators = pd.read_csv('data/dhis2/health_indicators/health_indicators.csv',index_col=None)
health_indicators


,Health Indicators,IDs
0,01. Maternal Health,?dimension=dx:sjorO3gQABy;nHvaVDaRcQU;llFRYYZL...
1,02. Child Health,?dimension=dx:LwxiPaHQwif;IQ1WyeMu4PS;Pd6AH5ko...
2,03. Immunization,?dimension=dx:a2UI5138tBe;w0rSItvQO65;vHWCFOMx...
3,04. Newborn,?dimension=dx:f46jAdwYWDD;FzTEiMtiSQN;n8sW5SK2...
4,05. Logistics,?dimension=dx:t3oOvEIdZuw;gCIHrhpVM3p
5,06. Vaccine & Logistics stock of District,?dimension=dx:EI3rYMbue2l;x8f1fsbgspb;Hxwqpvzm...
6,07. Vaccine & Logistics stock of UpazilaMunCC,?dimension=dx:oWkwB8F0w3f;cdqjn347Zbe;MXblPCxm...
7,Antenatal Care (ANC),?dimension=dx:lqGV3xf4MnW;EMN9ubIGpGh;wpDTeNQUuEb


## Time Periods

In [41]:
years = {'2009':2009, '2010':2010, '2011':2011, '2012':2012,
         '2013':2013, '2014':2014, '2015':2015, '2016':2016,
         '2017':2017, '2018':2018,

        }
years

{'2009': 2009,
 '2010': 2010,
 '2011': 2011,
 '2012': 2012,
 '2013': 2013,
 '2014': 2014,
 '2015': 2015,
 '2016': 2016,
 '2017': 2017,
 '2018': 2018}

## Building the API queries


## Functions

In [42]:
def create_api_queries(indicators, period, geos, geo_types):
    api_queries = {}
    for i in range(len(indicators)):
        for year_item, year_value in period.items():
            for geo_item, geo_value in geos.items():
                dhis_main = 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/29/analytics.csv'
                dataElementGroups_variables= indicators['IDs'][i]
                geo_variables = f'&dimension=ou:OU_{geo_types[geo_item]}-{geo_value};'
                time_variables = f'dNLjKwsVjod&filter=pe:{year_item};LAST_12_MONTHS&displayProperty=NAME&outputIdScheme='
                url = dhis_main + dataElementGroups_variables + geo_variables + time_variables
                file_name = indicators['Health Indicators'][i] + '_' + year_item + '_' + geo_item
                file_name = file_name.replace(" ", "").replace(".", "")
                api_queries[file_name] = url
    return api_queries

def data_download(api_queries, pause_length, dest_dir):
    for file_name, api_query in api_queries.items():
        print(f"Starting the process for {file_name} files: with geo codes and names respectively")
        print(f"Constructing file path for {file_name} file")
        file_with_codes = os.path.join(dest_dir, file_name + '_CODE' + '.csv') 
        file_with_names = os.path.join(dest_dir, file_name + '_NAME' + '.csv') 
        if file_name + '_CODE' + '.csv' in os.listdir(dest_dir):
            print(f"{file_name} file already downloaded")
        else:
            with requests.Session() as s:
                print(f"Getting API query for {file_with_codes} file")
                data_codes = s.get(api_query + 'CODE', auth=('wbbigdata_view', 'DGHS@1234#'),stream=True)
                with open(file_with_codes, 'wb') as f:
                    print(file_with_codes)
                    print(f"Writing in chunks {file_with_codes} file")
                    for chunk in data_codes.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"Pausing the process for {pause_length} seconds")
                time.sleep(pause_length)
                
        if file_name + '_NAME' + '.csv' in os.listdir(dest_dir):
            print(f"{file_name} file already downloaded")
        else:
            with requests.Session() as s:
                print(f"Restarting the process for {file_with_names} file")
                data_names = s.get(api_query + 'NAME', auth=('wbbigdata_view', 'DGHS@1234#'),stream=True)
                with open(file_with_names, 'wb') as f:
                    print(f"Writing in chunks {file_with_names} file")
                    for chunk in data_names.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
            print(f"Pausing the process for {pause_length} seconds")
            time.sleep(pause_length)


### Periods

In [43]:
years

{'2009': 2009,
 '2010': 2010,
 '2011': 2011,
 '2012': 2012,
 '2013': 2013,
 '2014': 2014,
 '2015': 2015,
 '2016': 2016,
 '2017': 2017,
 '2018': 2018}

### Geos

In [44]:
geos

{'District': 'PyK69cVqr5l',
 'Upazila': 'XpE4AlQZRkV',
 'Upazila Health Complex': 'TwFxBSq8iAg'}

### Geo Types

In [45]:
geo_types

{'District': 'GROUP', 'Upazila': 'GROUP', 'Upazila Health Complex': 'GROUP'}

### Downloading data

In [46]:
health_indicators

,Health Indicators,IDs
0,01. Maternal Health,?dimension=dx:sjorO3gQABy;nHvaVDaRcQU;llFRYYZL...
1,02. Child Health,?dimension=dx:LwxiPaHQwif;IQ1WyeMu4PS;Pd6AH5ko...
2,03. Immunization,?dimension=dx:a2UI5138tBe;w0rSItvQO65;vHWCFOMx...
3,04. Newborn,?dimension=dx:f46jAdwYWDD;FzTEiMtiSQN;n8sW5SK2...
4,05. Logistics,?dimension=dx:t3oOvEIdZuw;gCIHrhpVM3p
5,06. Vaccine & Logistics stock of District,?dimension=dx:EI3rYMbue2l;x8f1fsbgspb;Hxwqpvzm...
6,07. Vaccine & Logistics stock of UpazilaMunCC,?dimension=dx:oWkwB8F0w3f;cdqjn347Zbe;MXblPCxm...
7,Antenatal Care (ANC),?dimension=dx:lqGV3xf4MnW;EMN9ubIGpGh;wpDTeNQUuEb


In [47]:
dghis_queries = create_api_queries(indicators=health_indicators, 
                                   period=years, 
                                   geos=geos, 
                                   geo_types=geo_types)   

In [48]:
dghis_queries

{'01MaternalHealth_2009_District': 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/29/analytics.csv?dimension=dx:sjorO3gQABy;nHvaVDaRcQU;llFRYYZLkur;IVpv2Tj5rbj;UeBuoOKVSt9;DA4SHBTcotS;EN6VvWHuz6H;suIr6bcCKVv;ejt6ia79ixB;IQEbbjwChym;CRstBHmZrRi;pqxYwxpF3yV;eykgM4I5jQF&dimension=ou:OU_GROUP-PyK69cVqr5l;dNLjKwsVjod&filter=pe:2009;LAST_12_MONTHS&displayProperty=NAME&outputIdScheme=',
 '01MaternalHealth_2009_Upazila': 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/29/analytics.csv?dimension=dx:sjorO3gQABy;nHvaVDaRcQU;llFRYYZLkur;IVpv2Tj5rbj;UeBuoOKVSt9;DA4SHBTcotS;EN6VvWHuz6H;suIr6bcCKVv;ejt6ia79ixB;IQEbbjwChym;CRstBHmZrRi;pqxYwxpF3yV;eykgM4I5jQF&dimension=ou:OU_GROUP-XpE4AlQZRkV;dNLjKwsVjod&filter=pe:2009;LAST_12_MONTHS&displayProperty=NAME&outputIdScheme=',
 '01MaternalHealth_2009_UpazilaHealthComplex': 'https://centraldhis.mohfw.gov.bd/dhismohfw/api/29/analytics.csv?dimension=dx:sjorO3gQABy;nHvaVDaRcQU;llFRYYZLkur;IVpv2Tj5rbj;UeBuoOKVSt9;DA4SHBTcotS;EN6VvWHuz6H;suIr6bcCKVv;ejt6ia79ixB;

In [49]:
len(dghis_queries)

240

In [51]:
data_download(api_queries=dghis_queries, pause_length=45, dest_dir='data/dhis2/health_indicators/')

Starting the process for 01MaternalHealth_2009_Upazila files: with geo codes and names respectively
Constructing file path for 01MaternalHealth_2009_Upazila file
01MaternalHealth_2009_Upazila file already downloaded
01MaternalHealth_2009_Upazila file already downloaded
Starting the process for 01MaternalHealth_2009_UpazilaHealthComplex files: with geo codes and names respectively
Constructing file path for 01MaternalHealth_2009_UpazilaHealthComplex file
01MaternalHealth_2009_UpazilaHealthComplex file already downloaded
01MaternalHealth_2009_UpazilaHealthComplex file already downloaded
Starting the process for 01MaternalHealth_2009_District files: with geo codes and names respectively
Constructing file path for 01MaternalHealth_2009_District file
01MaternalHealth_2009_District file already downloaded
01MaternalHealth_2009_District file already downloaded
Starting the process for 01MaternalHealth_2010_Upazila files: with geo codes and names respectively
Constructing file path for 01Mater

data/dhis2/health_indicators/03Immunization_2018_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/03Immunization_2018_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/03Immunization_2018_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/03Immunization_2018_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 03Immunization_2018_District files: with geo codes and names respectively
Constructing file path for 03Immunization_2018_District file
03Immunization_2018_District file already downloaded
03Immunization_2018_District file already downloaded
Starting the process for 04Newborn_2009_Upazila files: with geo codes and names respectively
Constructing file path for 04Newborn_2009_Upazila file
04Newborn_2009_Upazila file already downloaded
04Newborn_2009_Upazila file already downloaded
Starting the process for 04N

data/dhis2/health_indicators/04Newborn_2015_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/04Newborn_2015_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/04Newborn_2015_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/04Newborn_2015_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 04Newborn_2015_District files: with geo codes and names respectively
Constructing file path for 04Newborn_2015_District file
04Newborn_2015_District file already downloaded
04Newborn_2015_District file already downloaded
Starting the process for 04Newborn_2016_Upazila files: with geo codes and names respectively
Constructing file path for 04Newborn_2016_Upazila file
04Newborn_2016_Upazila file already downloaded
04Newborn_2016_Upazila file already downloaded
Starting the process for 04Newborn_2016_UpazilaHealthComplex files: 

data/dhis2/health_indicators/05Logistics_2012_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/05Logistics_2012_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/05Logistics_2012_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/05Logistics_2012_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 05Logistics_2012_District files: with geo codes and names respectively
Constructing file path for 05Logistics_2012_District file
05Logistics_2012_District file already downloaded
05Logistics_2012_District file already downloaded
Starting the process for 05Logistics_2013_Upazila files: with geo codes and names respectively
Constructing file path for 05Logistics_2013_Upazila file
05Logistics_2013_Upazila file already downloaded
05Logistics_2013_Upazila file already downloaded
Starting the process for 05Logistics_2013_Up

data/dhis2/health_indicators/06Vaccine&LogisticsstockofDistrict_2009_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/06Vaccine&LogisticsstockofDistrict_2009_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/06Vaccine&LogisticsstockofDistrict_2009_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/06Vaccine&LogisticsstockofDistrict_2009_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 06Vaccine&LogisticsstockofDistrict_2009_District files: with geo codes and names respectively
Constructing file path for 06Vaccine&LogisticsstockofDistrict_2009_District file
06Vaccine&LogisticsstockofDistrict_2009_District file already downloaded
06Vaccine&LogisticsstockofDistrict_2009_District file already downloaded
Starting the process for 06Vaccine&LogisticsstockofDistrict_2010_Upazila files: with geo codes and names respe

Writing in chunks data/dhis2/health_indicators/06Vaccine&LogisticsstockofDistrict_2014_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 06Vaccine&LogisticsstockofDistrict_2014_District files: with geo codes and names respectively
Constructing file path for 06Vaccine&LogisticsstockofDistrict_2014_District file
06Vaccine&LogisticsstockofDistrict_2014_District file already downloaded
06Vaccine&LogisticsstockofDistrict_2014_District file already downloaded
Starting the process for 06Vaccine&LogisticsstockofDistrict_2015_Upazila files: with geo codes and names respectively
Constructing file path for 06Vaccine&LogisticsstockofDistrict_2015_Upazila file
06Vaccine&LogisticsstockofDistrict_2015_Upazila file already downloaded
06Vaccine&LogisticsstockofDistrict_2015_Upazila file already downloaded
Starting the process for 06Vaccine&LogisticsstockofDistrict_2015_UpazilaHealthComplex files: with geo codes and names respectively
Constructing file path f

data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2010_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2010_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2010_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2010_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 07Vaccine&LogisticsstockofUpazilaMunCC_2010_District files: with geo codes and names respectively
Constructing file path for 07Vaccine&LogisticsstockofUpazilaMunCC_2010_District file
07Vaccine&LogisticsstockofUpazilaMunCC_2010_District file already downloaded
07Vaccine&LogisticsstockofUpazilaMunCC_2010_District file already downloaded
Starting the process for 07Vaccine&LogisticsstockofUpazilaMunCC_2011_Upazila f

data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2015_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2015_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2015_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/07Vaccine&LogisticsstockofUpazilaMunCC_2015_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for 07Vaccine&LogisticsstockofUpazilaMunCC_2015_District files: with geo codes and names respectively
Constructing file path for 07Vaccine&LogisticsstockofUpazilaMunCC_2015_District file
07Vaccine&LogisticsstockofUpazilaMunCC_2015_District file already downloaded
07Vaccine&LogisticsstockofUpazilaMunCC_2015_District file already downloaded
Starting the process for 07Vaccine&LogisticsstockofUpazilaMunCC_2016_Upazila f

Starting the process for AntenatalCare(ANC)_2010_District files: with geo codes and names respectively
Constructing file path for AntenatalCare(ANC)_2010_District file
AntenatalCare(ANC)_2010_District file already downloaded
AntenatalCare(ANC)_2010_District file already downloaded
Starting the process for AntenatalCare(ANC)_2011_Upazila files: with geo codes and names respectively
Constructing file path for AntenatalCare(ANC)_2011_Upazila file
AntenatalCare(ANC)_2011_Upazila file already downloaded
AntenatalCare(ANC)_2011_Upazila file already downloaded
Starting the process for AntenatalCare(ANC)_2011_UpazilaHealthComplex files: with geo codes and names respectively
Constructing file path for AntenatalCare(ANC)_2011_UpazilaHealthComplex file
Getting API query for data/dhis2/health_indicators/AntenatalCare(ANC)_2011_UpazilaHealthComplex_CODE.csv file
data/dhis2/health_indicators/AntenatalCare(ANC)_2011_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/Antenata

data/dhis2/health_indicators/AntenatalCare(ANC)_2017_UpazilaHealthComplex_CODE.csv
Writing in chunks data/dhis2/health_indicators/AntenatalCare(ANC)_2017_UpazilaHealthComplex_CODE.csv file
Pausing the process for 45 seconds
Restarting the process for data/dhis2/health_indicators/AntenatalCare(ANC)_2017_UpazilaHealthComplex_NAME.csv file
Writing in chunks data/dhis2/health_indicators/AntenatalCare(ANC)_2017_UpazilaHealthComplex_NAME.csv file
Pausing the process for 45 seconds
Starting the process for AntenatalCare(ANC)_2017_District files: with geo codes and names respectively
Constructing file path for AntenatalCare(ANC)_2017_District file
AntenatalCare(ANC)_2017_District file already downloaded
AntenatalCare(ANC)_2017_District file already downloaded
Starting the process for AntenatalCare(ANC)_2018_Upazila files: with geo codes and names respectively
Constructing file path for AntenatalCare(ANC)_2018_Upazila file
AntenatalCare(ANC)_2018_Upazila file already downloaded
AntenatalCare(AN